# Очищаем триплеты от мусора

In [ ]:
import os
import re
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import logging

In [ ]:
# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)

# Регулярное выражение для проверки допустимых символов (английские буквы, цифры, дефис, +-=_%)
allowed_pattern = re.compile(r"^[a-zA-Z0-9\-\+\=\%\_]+$")

# Функция для проверки строки на наличие недопустимых символов
def contains_invalid_chars(text):
    return not bool(allowed_pattern.match(text))

# Функция для лемматизации текста
def lemmatize_text(text):
    # Удаляем стоп-слова и лемматизируем каждое слово
    stop_words = set(stopwords.words('english'))
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    return " ".join(lemmatized_words)

# Функция для обработки одной строки датафрейма
def process_row(row, invalid_count):
    processed_row = []
    for item in row:
        if contains_invalid_chars(item):
            invalid_count += 1
            processed_row.append("")  # Заменяем недопустимую строку на пустую
        else:
            processed_row.append(lemmatize_text(item))
    return processed_row, invalid_count

# Функция для обработки одного файла
def process_file(input_file_path, output_file_path):
    invalid_count = 0
    try:
        # Чтение файла как CSV с разделителем ";"
        df = pd.read_csv(input_file_path, sep=";", header=None)
        
        # Обработка каждой строки
        processed_data = []
        for _, row in df.iterrows():
            processed_row, invalid_count = process_row(row, invalid_count)
            processed_data.append(processed_row)
        
        # Создание нового датафрейма из обработанных данных
        processed_df = pd.DataFrame(processed_data)
        
        # Сохранение обработанного файла
        processed_df.to_csv(output_file_path, sep=";", index=False, header=False)
        
        return len(df), invalid_count, None  # Возвращаем None, если ошибок нет
    except Exception as e:
        error_message = f"Ошибка при обработке файла {input_file_path}: {e}"
        logging.error(error_message)
        return 0, invalid_count, input_file_path  # Возвращаем путь к файлу с ошибкой

# Основная функция для обработки всех файлов в директории
def process_directory(input_directory_path, output_directory_path):
    input_directory = Path(input_directory_path)
    output_directory = Path(output_directory_path)
    
    # Создаем выходную директорию, если она не существует
    output_directory.mkdir(parents=True, exist_ok=True)
    
    total_files = sum(1 for _ in input_directory.rglob("*.txt"))  # Подсчет всех txt файлов
    total_rows_processed = 0
    total_invalid_rows = 0
    error_files = []  # Список для хранения путей файлов с ошибками
    
    # Итерация по всем файлам в директории с прогресс-баром
    with tqdm(total=total_files, desc="Обработка файлов") as pbar:
        for input_file_path in input_directory.rglob("*.txt"):
            # Генерация пути для выходного файла
            relative_path = input_file_path.relative_to(input_directory)
            output_file_path = output_directory / relative_path
            output_file_path.parent.mkdir(parents=True, exist_ok=True)  # Создаем поддиректории
            
            rows_processed, invalid_rows, error_file = process_file(input_file_path, output_file_path)
            total_rows_processed += rows_processed
            total_invalid_rows += invalid_rows
            if error_file:
                error_files.append(error_file)  # Добавляем путь файла с ошибкой
            pbar.update(1)
            pbar.set_postfix({
                "Обработано строк": total_rows_processed,
                "Удалено строк": total_invalid_rows
            })
    
    # Логирование файлов с ошибками
    if error_files:
        logging.warning(f"Обработка завершена, но возникли ошибки в следующих файлах:")
        for error_file in error_files:
            logging.warning(error_file)
    else:
        logging.info("Обработка завершена без ошибок.")
    
    print(f"Обработка завершена. Всего обработано строк: {total_rows_processed}, удалено строк: {total_invalid_rows}")

In [ ]:
# Вызов основной функции
input_directory = Path("/home/kdemyokhin_1/concept-tree-course-work/articles_triples/arxiv-txt-cs")
output_directory = Path("/home/kdemyokhin_1/concept-tree-course-work/articles_triples_cleaned/arxiv-txt-cs")
process_directory(input_directory, output_directory)